In [1]:
print("start import")
import time
time1 = time.time()
import pandas as pd
import torch
from torch_geometric.data import HeteroData
import os
time2 = time.time()
print("import time: ", time2 - time1)
print(f"当前时间: {time.strftime('%m-%d %H:%M:%S', time.localtime())}")

start import
import time:  7.9205241203308105
当前时间: 11-08 23:03:02


In [24]:
print("start read data")
time1 = time.time()
path = "D:\BTNHG\BTNHGV2"
# 1. 读取三个特征文件和一个边文件
# 使用os.path.join确保跨平台兼容性
addr_feat_df = pd.read_csv(os.path.join(path, "addressFeature.csv"))
coin_feat_df = pd.read_csv(os.path.join(path, "coinFeature.csv"))
tx_feat_df = pd.read_csv(os.path.join(path, "TxFeature.csv"))
edge_df = pd.read_csv(os.path.join(path, "hgEdgeV2.csv"))
time2 = time.time()
print(f"读取数据时间: {time2 - time1}")
print(f"当前时间: {time.strftime('%m-%d %H:%M:%S', time.localtime())}")

start read data
读取数据时间: 3.7070651054382324
当前时间: 11-09 00:06:35


In [27]:
# 2. 建立 ID 映射（保证 PyG 节点索引连续）
address_ids = addr_feat_df['addressID'].unique()
coin_ids    = coin_feat_df['coinID'].unique()
tx_ids      = tx_feat_df['txID'].unique()

address_id_map = {id_: i for i, id_ in enumerate(address_ids)}
coin_id_map    = {id_: i for i, id_ in enumerate(coin_ids)}
tx_id_map      = {id_: i for i, id_ in enumerate(tx_ids)}


In [30]:
# 3. 初始化 HeteroData
data = HeteroData()

# 4. 构建节点特征矩阵
# 去掉 ID 列，只保留数值特征
addr_features = addr_feat_df.drop(columns=['addressID']).values.astype(float)
coin_features = coin_feat_df.drop(columns=['coinID']).values.astype(float)
tx_features   = tx_feat_df.drop(columns=['txID']).values.astype(float)



In [ ]:
# 转换为 torch.tensor，并对齐索引
# 优化前
address_x = torch.zeros((len(address_ids), addr_features.shape[1]))
for _, row in addr_feat_df.iterrows():
    idx = address_id_map[row['addressID']]
    address_x[idx] = torch.tensor(row.drop(labels=['addressID']).values, dtype=torch.float)




In [32]:
#打印address_x形状
print(address_x.shape)

torch.Size([546649, 13])


In [35]:
import numpy as np
#获取所有addressID对应的索引
address_indices = addr_feat_df['addressID'].map(address_id_map).values

# 提取特征数据并转换为numpy数组
feature_data = addr_feat_df.drop(columns=['addressID']).values.astype(np.float32)

# 一次性创建目标张量
address_x2 = torch.zeros((len(address_ids), addr_features.shape[1]), dtype=torch.float32)

# 批量赋值（比逐行赋值快得多）
address_x2[address_indices] = torch.from_numpy(feature_data)


In [43]:
def tensor_equal_with_nan(a, b):
    # 逐元素比较：要么值相等，要么同时为 NaN
    return torch.all((a == b) | (torch.isnan(a) & torch.isnan(b)))

print(tensor_equal_with_nan(address_x, address_x2))


tensor(True)
